In [1]:
%pylab
from glob import glob
from natsort import natsorted
import h5py
from scipy.stats import binned_statistic
import palettable.colorbrewer.qualitative
from matplotlib.ticker import ScalarFormatter, NullFormatter,FormatStrFormatter
import pylab

from palettable.colorbrewer.qualitative import Dark2_7
#from palettable.cubehelix import Cubehelix
#palette = Cubehelix.make(n=10)

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


# big multi-panel plot

In [ ]:
plt.clf()
fig, ax = plt.subplots(1,4,figsize=(8,1.6))
fig.delaxes(ax[3])
dirs = natsorted(glob("sinkdata/*/stars"))
q = np.array([0,14,50,86,100])

#for a in ax.flatten():
    #a.set_prop_cycle('color', Dark2_7.mpl_colors)

M0 = 2e4

labeldict = {"sinkdata/b_default_n_n/stars": r"TURBSPHERE",
             "sinkdata/b_default_y_n/stars": r"TURBSPHERE+Driving",
             "sinkdata/b_default_n_y/stars": r"TURBSPHERE+Jets",
             "sinkdata/b_default_y_y/stars": r"TURBSPHERE+Driving+Jets",
             "sinkdata/box/stars": r"BOX+Driving",
             "sinkdata/box_jets/stars": r"BOX+Driving+Jets",
             "sinkdata/sphere/stars": r"SPHERE",
             "sinkdata/sphere_jets/stars": r"SPHERE+Jets",
            }

plt.rcParams.update({'axes.labelsize': 6})
#plt.rcParams.update({'axes.ticklabelsize': 8})

plt.rc('xtick',labelsize=6)
plt.rc('ytick',labelsize=6)

i = 0
for d in dirs:
    print(d)
    name = d
    if not d in labeldict.keys(): continue

    if glob(d+"/snapshot*stars.hdf5"):
        files = natsorted(glob(d+"/snapshot*stars.hdf5"))
    else:
        continue

    quantiles = []
    means = []
    mquantiles = []
    mtot = []
    times = []
    Nsink = []
    machs = []
    mpeak = []
    

    for f in files:
        #print(f)
        with h5py.File(f,'r') as F:
            if "PartType5" in F.keys(): 
                mstar = np.array(F["PartType5"]["Masses"])
                mstar = mstar[mstar>1e-2]
            else:
                mstar = np.zeros(1)
            means.append(mstar.mean())
            quantiles.append(np.percentile(mstar, q))
            mquantiles.append(np.interp(np.log10(mstar.sum()*q/100), np.log10(np.sort(mstar).cumsum()), np.sort(mstar)))
            mtot.append(mstar.sum())

            times.append(F["Header"].attrs["Time"]*979)
            Nsink.append(len(mstar))
            machs.append(np.array(F["Mach"]))
            #print(F["Header"].attrs.keys())
            npeak = np.array([np.sum(np.abs(np.log10(mm/mstar))<0.1) for mm in mstar])
            mpeak.append(mstar[npeak.argmax()])
    quantiles = np.array(quantiles)
    mquantiles = np.array(mquantiles)
    means = np.array(means)
    mtot = np.array(mtot)
    times = np.array(times)
    Nsink = np.array(Nsink)
    mpeak = np.array(mpeak)
    print(mtot.max())
    #ls = ('solid' if i < 7 else 'dashed')
    if "b_default" in d: ls='solid'
    elif "box" in d: ls='dashed'
    else: ls='dashdot'
    
    driving = '#cf1578'#cf1578, #e8d21d, #039fbe and #b20238
    jets = '#9bc472'
    drivingjets = '#039fbe'
    none = '#f57e7e'
    lwd=1
    if "b_default" in d:
        if "n_n" in d: color=none; lwd=1.25
        elif "n_y" in d: color=jets; lwd=1
        elif "y_n" in d: color=driving; lwd=1.25
        elif "y_y" in d: color=drivingjets; lwd=1
    elif "box" in d:
        if "box_jets" in d: color=drivingjets; lwd=1
        else: color = driving; lwd=1.25
    elif "sphere" in d:
        if "sphere_jets" in d: color=jets; lwd=1
        else: color=none; lwd=1.25
    #times -= times[Nsink>1][0]
    ax[0].plot(times, mtot/M0, label=labeldict[name], color=color,ls=ls,zorder=i,lw=lwd)
    ax[1].plot(times, Nsink,lw=lwd,ls=ls,zorder=i,color=color)
    ax[2].plot(times, machs,zorder=i,lw=lwd,ls=ls,color=color)
    i += 1
Tmax=12
ax[0].set(yscale='log',xlim=[1,Tmax],ylim=[3e-4,0.3], ylabel="SFE",xlabel=r"$t$ (Myr)",xscale='log')
ax[1].set(yscale='log',xscale='log',ylim=[1,8000], xlim=[1,Tmax],ylabel="$N_\mathrm{*}$",xlabel=r"$t$ (Myr)")

ax[2].set(yscale='log',xscale='log', xlim=[1,Tmax], ylim=[7,30],xlabel=r"$t$ (Myr)",ylabel="$\mathcal{M}$")

#ax[0].legend(labelspacing=0,fontsize=7,bbox_to_anchor=(5.3,1),bbox_transform=ax[0].transAxes, ncol=1)

fig.subplots_adjust(hspace=0.3,wspace=0.35)
plt.savefig("M2e4_Turbsphere_SFE_BIG.pdf",bbox_inches='tight')

In [ ]:
 #   if "b_default" in d: ls='solid'
 #   elif "box" in d: ls='dashed'
 #   else: ls='dashdot'
#labeldict = {"sinkdata/b_default_n_n/stars": r"TURBSPHERE",
#             "sinkdata/b_default_y_n/stars": r"TURBSPHERE+Driving",
#             "sinkdata/b_default_n_y/stars": r"TURBSPHERE+Jets",
#             "sinkdata/b_default_y_y/stars": r"TURBSPHERE+Driving+Jets",
#             "sinkdata/box/stars": r"BOX+Driving",
#             "sinkdata/box_jets/stars": r"BOX+Driving+Jets",
#             "sinkdata/sphere/stars": r"SPHERE",
#             "sinkdata/sphere_jets/stars": r"SPHERE+Jets",
#            }
plt.clf()

driving = '#cf1578'#cf1578, #e8d21d, #039fbe and #b20238
jets = '#9bc472'
drivingjets = '#039fbe'
none = '#f57e7e'
import pylab
x = np.linspace(1, 100, 1000)
y = np.log(x)
y1 = np.sin(x)
fig = plt.figure("Line plot")
legendFig = plt.figure("Legend plot")
ax = fig.add_subplot(111)
line1, = ax.plot(x, y, c=none, lw=1.25, linestyle="solid")
line2, = ax.plot(x, y1, c=jets, lw=1, linestyle="solid")
legendFig.legend([line1, line2], [r"TURBSPHERE", r"TURBSPHERE+Jets"], loc='center')
legendFig.savefig('legend0.pdf', bbox_inches='tight')

In [ ]:
 #   if "b_default" in d: ls='solid'
 #   elif "box" in d: ls='dashed'
 #   else: ls='dashdot'
#labeldict = {"sinkdata/b_default_n_n/stars": r"TURBSPHERE",
#             "sinkdata/b_default_y_n/stars": r"TURBSPHERE+Driving",
#             "sinkdata/b_default_n_y/stars": r"TURBSPHERE+Jets",
#             "sinkdata/b_default_y_y/stars": r"TURBSPHERE+Driving+Jets",
#             "sinkdata/box/stars": r"BOX+Driving",
#             "sinkdata/box_jets/stars": r"BOX+Driving+Jets",
#             "sinkdata/sphere/stars": r"SPHERE",
#             "sinkdata/sphere_jets/stars": r"SPHERE+Jets",
#            }
plt.clf()

driving = '#cf1578'#cf1578, #e8d21d, #039fbe and #b20238
jets = '#9bc472'
drivingjets = '#039fbe'
none = '#f57e7e'
import pylab
x = np.linspace(1, 100, 1000)
y = np.log(x)
y1 = np.sin(x)
fig = plt.figure("Line plot")
legendFig = plt.figure("Legend plot")
ax = fig.add_subplot(111)
line1, = ax.plot(x, y, c=driving, lw=1.25, linestyle="solid")
line2, = ax.plot(x, y1, c=drivingjets, lw=1, linestyle="solid")
legendFig.legend([line1, line2], [r"TURBSPHERE+Driving", r"TURBSPHERE+Driving+Jets"], loc='center')
legendFig.savefig('legend1.pdf', bbox_inches='tight')

In [ ]:
 #   if "b_default" in d: ls='solid'
 #   elif "box" in d: ls='dashed'
 #   else: ls='dashdot'
#labeldict = {"sinkdata/b_default_n_n/stars": r"TURBSPHERE",
#             "sinkdata/b_default_y_n/stars": r"TURBSPHERE+Driving",
#             "sinkdata/b_default_n_y/stars": r"TURBSPHERE+Jets",
#             "sinkdata/b_default_y_y/stars": r"TURBSPHERE+Driving+Jets",
#             "sinkdata/box/stars": r"BOX+Driving",
#             "sinkdata/box_jets/stars": r"BOX+Driving+Jets",
#             "sinkdata/sphere/stars": r"SPHERE",
#             "sinkdata/sphere_jets/stars": r"SPHERE+Jets",
#            }
plt.clf()

driving = '#cf1578'#cf1578, #e8d21d, #039fbe and #b20238
jets = '#9bc472'
drivingjets = '#039fbe'
none = '#f57e7e'
import pylab
x = np.linspace(1, 100, 1000)
y = np.log(x)
y1 = np.sin(x)
fig = plt.figure("Line plot")
legendFig = plt.figure("Legend plot")
ax = fig.add_subplot(111)
line1, = ax.plot(x, y, c=driving, lw=1.25, linestyle="dashed")
line2, = ax.plot(x, y1, c=drivingjets, lw=1, linestyle="dashed")
legendFig.legend([line1, line2], [r"BOX+Driving", r"BOX+Driving+Jets"], loc='center')
legendFig.savefig('legend2.pdf', bbox_inches='tight')

In [2]:
 #   if "b_default" in d: ls='solid'
 #   elif "box" in d: ls='dashed'
 #   else: ls='dashdot'
#labeldict = {"sinkdata/b_default_n_n/stars": r"TURBSPHERE",
#             "sinkdata/b_default_y_n/stars": r"TURBSPHERE+Driving",
#             "sinkdata/b_default_n_y/stars": r"TURBSPHERE+Jets",
#             "sinkdata/b_default_y_y/stars": r"TURBSPHERE+Driving+Jets",
#             "sinkdata/box/stars": r"BOX+Driving",
#             "sinkdata/box_jets/stars": r"BOX+Driving+Jets",
#             "sinkdata/sphere/stars": r"SPHERE",
#             "sinkdata/sphere_jets/stars": r"SPHERE+Jets",
#            }
plt.clf()
driving = '#cf1578'#cf1578, #e8d21d, #039fbe and #b20238
jets = '#9bc472'
drivingjets = '#039fbe'
none = '#f57e7e'
import pylab
x = np.linspace(1, 100, 1000)
y = np.log(x)
y1 = np.sin(x)
fig = plt.figure("Line plot")
legendFig = plt.figure("Legend plot")
ax = fig.add_subplot(111)
line1, = ax.plot(x, y, c=none, lw=1.25, linestyle="dashdot")
line2, = ax.plot(x, y1, c=jets, lw=1, linestyle="dashdot")
legendFig.legend([line1, line2], [r"SPHERE", r"SPHERE+Jets"], loc='center')
legendFig.savefig('legend3.pdf', bbox_inches='tight')

findfont: Font family ['serif'] not found. Falling back to DejaVu Sans.


## Just SF histories

In [88]:
plt.clf()
fig, ax = plt.subplots(1,1,figsize=(4,4))
#fig.delaxes(ax[0,3])
dirs = natsorted(glob("sinkdata/*/stars"))
q = np.array([0,14,50,86,100])


ax.set_prop_cycle('color', Dark2_7.mpl_colors)

M0 = 2e4

labeldict = {"sinkdata/b_default_n_n/stars": r"TURBSPHERE",
             "sinkdata/b_default_y_n/stars": r"TURBSPHERE+Driving",
             "sinkdata/b_default_n_y/stars": r"TURBSPHERE+Jets",
             "sinkdata/b_default_y_y/stars": r"TURBSPHERE+Driving+Jets",
             "sinkdata/box/stars": r"BOX+Driving",
             "sinkdata/box_jets/stars": r"BOX+Driving+Jets",
             "sinkdata/sphere/stars": r"SPHERE",
             "sinkdata/sphere_jets/stars": r"SPHERE+Jets",
            }

#plt.rcParams.update({'axes.labelsize': 6})
#plt.rcParams.update({'axes.ticklabelsize': 8})

#plt.rc('xtick',labelsize=6)
#plt.rc('ytick',labelsize=6)

i = 0
for d in dirs:
    print(d)
    name = d
    if not d in labeldict.keys(): continue

    if glob(d+"/snapshot*stars.hdf5"):
        files = natsorted(glob(d+"/snapshot*stars.hdf5"))
    else:
        continue

    quantiles = []
    means = []
    mquantiles = []
    mtot = []
    times = []
    Nsink = []
    machs = []
    mpeak = []
    SFE = []

    for f in files:
        #print(f)
        with h5py.File(f,'r') as F:
            if "PartType5" in F.keys(): 
                mstar = np.array(F["PartType5"]["Masses"])
    #            if not (mstar>0.08).sum(): continue
                mstar = mstar[mstar>1e-2]
            else:
                mstar = np.zeros(1)
            mtot.append(mstar.sum())
            Nsink.append(len(mstar))
            times.append(F["Header"].attrs["Time"]*979)
    
    mtot = np.array(mtot)
    times = np.array(times)
    Nsink = np.array(Nsink)
    
    if "b_default" in d: ls='solid'
    elif "box" in d: ls='dashed'
    else: ls='dashdot'
        
    times -= times[Nsink>1][0]
    ax.plot(times, mtot/M0, label=labeldict[name], ls=ls,lw=1)#,label="$\Delta m{=}%sM_\odot$"%dm,lw=1)
    SFE.append(mtot/M0)
    print((np.array(SFE)[-1])[-1])
    i += 1
Tmax=12
ax.set(yscale='log',xlim=[0.1,Tmax],ylim=[1/2e4,0.3], ylabel="SFE",xlabel=r"$t$ (Myr)",xscale='log')

ax.legend(labelspacing=0,fontsize=7)#,bbox_to_anchor=(5.3,1),bbox_transform=ax[0,0].transAxes, ncol=1)

#fig.subplots_adjust(hspace=0.3,wspace=0.35)
plt.savefig("M2e4_Turbsphere_SFE.pdf",bbox_inches='tight')

sinkdata/b_default_n_n/stars
0.2169349609375
sinkdata/b_default_n_y/stars
0.11004031982421875
sinkdata/b_default_y_n/stars
0.1549457763671875
sinkdata/b_default_y_y/stars
0.06159051513671875
sinkdata/box/stars
0.07102557373046875
sinkdata/box_jets/stars
0.014723760986328126
sinkdata/sphere/stars
0.2921966552734375
sinkdata/sphere_jets/stars
0.106851171875


# IMF

In [110]:
plt.clf()
fig, ax = plt.subplots(1,1,figsize=(4,4))
#fig.delaxes(ax[0,3])
dirs = natsorted(glob("sinkdata/*/stars"))
q = np.array([0,14,50,86,100])
mgrid = np.logspace(-2,2,1000)
alpha3 = -2.3
limit = 0.1



M0 = 2e4

labeldict = {"sinkdata/b_default_n_n/stars": r"TURBSPHERE",
             "sinkdata/b_default_y_n/stars": r"TURBSPHERE+Driving",
             "sinkdata/b_default_n_y/stars": r"TURBSPHERE+Jets",
             "sinkdata/b_default_y_y/stars": r"TURBSPHERE+Driving+Jets",
             "sinkdata/box/stars": r"BOX+Driving",
             "sinkdata/box_jets/stars": r"BOX+Driving+Jets",
             "sinkdata/sphere/stars": r"SPHERE",
             "sinkdata/sphere_jets/stars": r"SPHERE+Jets",
            }

i = 0
for d in dirs:
    print(d)
    name = d
    if not d in labeldict.keys(): continue

    if glob(d+"/snapshot*stars.hdf5"):
        files = natsorted(glob(d+"/snapshot*stars.hdf5"))
        lastsnap = natsorted(glob(d+"/snapshot*stars.hdf5"))[-1]

    else:
        continue

    quantiles = []
    means = []
    mquantiles = []
    mtot = []
    times = []
    Nsink = []
    machs = []
    mpeak = []
    

    with h5py.File(lastsnap,'r') as F:
        if "PartType5" in F.keys(): 
            mstar = np.array(F["PartType5"]["Masses"])
     #       print(mstar.max())
    #            if not (mstar>0.08).sum(): continue
            mstar = mstar[mstar>1e-2]
        else:
            mstar = np.zeros(1)                
        mtot.append(mstar.sum())

        times.append(F["Header"].attrs["Time"]*979)
        Nsink.append(len(mstar))

    mtot = np.array(mtot)
    times = np.array(times)
    Nsink = np.array(Nsink)
    if "b_default" in d: ls='solid'
    elif "box" in d: ls='dashed'
    else: ls='dashdot'
    
    driving = '#cf1578'#cf1578, #e8d21d, #039fbe and #b20238
    jets = '#9bc472'
    drivingjets = '#039fbe'
    none = '#f57e7e'
    if "b_default" in d:
        if "n_n" in d: color=none
        elif "n_y" in d: color=jets
        elif "y_n" in d: color=driving
        elif "y_y" in d: color=drivingjets
    elif "box" in d:
        if "box_jets" in d: color=drivingjets
        else: color = driving
    elif "sphere" in d:
        if "sphere_jets" in d: color=jets
        else: color=none
    times -= times[Nsink>1][0]
    mstar = mstar
    mass_bins = np.logspace(1,8,15)#############
    m_avg = (mass_bins[1:]*mass_bins[:-1])**0.5 
    print(np.sum(mstar))
    #print(mstar.min())
    ax.plot(mass_bins[1:], np.histogram(mstar,mass_bins)[0]/mtot, label=labeldict[name], color=color, ls=ls,lw=1)#,label="$\Delta m{=}%sM_\odot$"%dm,lw=1)
    i += 1
kroupasum=54.603034013490664*10**0
kroupa = np.empty_like(mgrid)
kroupa[mgrid<0.08] = (mgrid**-0.3 * mgrid)[mgrid < 0.08] * (kroupasum**-1)
kroupa[(mgrid>=0.08) * (mgrid < 0.5)] = (0.08**-0.3 * (mgrid/0.08)**-1.3 * mgrid)[(mgrid>=0.08) * (mgrid < 0.5)] * (kroupasum**-1)
kroupa[(mgrid >= 0.5)] = (0.08**-0.3 * (0.5/0.08)**-1.3 * (mgrid/0.5)**alpha3 * mgrid)[(mgrid >= 0.5)] * (kroupasum**-1)
#kroupa *= np.sum(mstar>limit) / np.sum(np.interp(m_avg,mgrid,kroupa*(mgrid>limit))) # normalize by number - change to mass?
kroupa[(mgrid>=0.08) * (mgrid < 0.5)] = (0.08**-0.3 * (mgrid/0.08)**-1.3 * mgrid)[(mgrid>=0.08) * (mgrid < 0.5)] * (kroupasum**-1)
kroupa[(mgrid >= 0.5)] = (0.08**-0.3 * (0.5/0.08)**-1.3 * (mgrid/0.5)**-2.3 * mgrid)[(mgrid >= 0.5)] * (kroupasum**-1)
kroupa *= 7.5*10**5#/#kroupa.sum()
print(kroupa.sum())
Tmax=12
ax.set(yscale='log', ylabel="$\mathrm{d}N/\mathrm{d} \log M_\star$",xlabel="Stellar Mass ($M_\odot$)",xscale='log', xlim=(10**-7,10**7), ylim=(5*10**-1, 2*10**4))
ax.plot(mgrid*10**3.4, kroupa, label="Kroupa 2001",ls='dashed',color='dimgray',zorder=-1)

ax.legend(labelspacing=0,fontsize=7,loc=1)#,bbox_to_anchor=(5.3,1),bbox_transform=ax[0,0].transAxes, ncol=1)

#fig.subplots_adjust(hspace=0.3,wspace=0.35)
plt.savefig("M2e4_Turbsphere_IMF.pdf",bbox_inches='tight')

sinkdata/b_default_n_n/stars
4338.699
sinkdata/b_default_n_y/stars
2200.8064
sinkdata/b_default_y_n/stars
3098.9155
sinkdata/b_default_y_y/stars
1231.8103
sinkdata/box/stars
1420.5115
sinkdata/box_jets/stars
294.47522
sinkdata/sphere/stars
5843.933
sinkdata/sphere_jets/stars
2137.0234
750000.0


In [90]:
masses = np.array([max_mass[i] for i in max_mass.keys()]) #np.array(h5py.File(f,'r')["PartType5"]["BH_Mass"]) 

mbins = np.logspace(-2,2.333,31)
m_avg = (mbins[1:]*mbins[:-1])**0.5

limit = 0.1

mgrid = np.logspace(-2,2,1000)
alpha3 = -2.3
#kroupa = np.empty_like(mgrid)
#kroupa[mgrid<0.08] = (mgrid**-0.3 * mgrid)[mgrid < 0.08]
#kroupa[(mgrid>=0.08) * (mgrid < 0.5)] = (0.08**-0.3 * (mgrid/0.08)**-1.3 * mgrid)[(mgrid>=0.08) * (mgrid < 0.5)]
#kroupa[(mgrid >= 0.5)] = (0.08**-0.3 * (0.5/0.08)**-1.3 * (mgrid/0.5)**alpha3 * mgrid)[(mgrid >= 0.5)]
#kroupa *= np.sum(masses>limit) / np.sum(np.interp(m_avg,mgrid,kroupa*(mgrid>limit))) # normalize by number - change to mass?

#chabrier = 0.28263 * mgrid**-2.3
#chabrier[mgrid<1] = ( 1/mgrid * np.exp(-np.log10(mgrid/0.08)**2 / (2*0.69**2)))[mgrid<1]
#chabrier *= mgrid
#chabrier *= np.sum(masses>limit) / np.sum(np.interp(m_avg,mgrid,chabrier*(mgrid>limit)))
#chabrier_sys = np.exp(-np.log10(1/0.22)**2 / (2*0.57**2)) * mgrid**-2.3
#chabrier_sys[mgrid<1] = ( 1/mgrid * np.exp(-np.log10(mgrid/0.22)**2 / (2*0.57**2)))[mgrid<1]
#chabrier_sys *= mgrid
#chabrier_sys *= np.sum(masses>limit) / np.sum(np.interp(m_avg,mgrid,chabrier_sys*(mgrid>limit)))
#kroupa[(mgrid>=0.08) * (mgrid < 0.5)] = (0.08**-0.3 * (mgrid/0.08)**-1.3 * mgrid)[(mgrid>=0.08) * (mgrid < 0.5)]
#kroupa[(mgrid >= 0.5)] = (0.08**-0.3 * (0.5/0.08)**-1.3 * (mgrid/0.5)**-2.3 * mgrid)[(mgrid >= 0.5)]
#parravano = mgrid * mgrid**-2.35 * (1 - np.exp(-(mgrid/0.35)**(0.51+1.35)))
#parravano *= np.sum(masses>limit) / np.sum(np.interp(m_avg,mgrid,parravano*(mgrid>limit)))


fig, ax = plt.subplots(figsize=(4,4))

counts = np.histogram(masses, bins=mbins)[0]
errors = counts**0.5
ax.hist(masses,mbins,histtype="stepfilled",facecolor="none",edgecolor="black")

#ax.hist(Mbound,mbins,histtype="stepfilled",facecolor="none",edgecolor="black")

#ax.errorbar(m_avg,counts,errors,ls='none',marker=None,capsize=4,color='black')

#counts = np.histogram(system_mass, bins=mbins)[0]
#errors = counts**0.5
#ax.hist(system_mass,mbins,histtype="stepfilled",facecolor="none",edgecolor="grey")
#ax.errorbar(m_avg,counts,errors,ls='none',marker=None,capsize=4,color='grey')

ax.set(xscale='log',yscale='log',xlabel="Initial Stellar Mass ($M_\odot$)", ylabel="Number of stars",xlim=[1e-2,3e2],ylim=[0.5,1e3])


#ax.plot(mgrid, kroupa, label="Kroupa 2001",ls='dashed',color='grey',zorder=-1)
#ax.plot(mgrid, chabrier, label="Chabrier 2003 (indiv.)",ls='dotted',color='grey',zorder=-1)
#ax.plot(mgrid, chabrier_sys, label="Chabrier 2003 (sys.)",ls='dotted',color='grey',zorder=-1)
#ax.plot(mgrid, parravano, label="Parravano 2011",ls='dashdot',color='grey',zorder=-1)


ax.text(0.03,3,"Incompleteness",rotation=90)

ax.fill_between([1e-3,0.1],[0,0],[1e6,1e6],color='grey',zorder=-1000,alpha=0.33)

ax.legend(loc=1)
plt.savefig("M2e4_full_IMF.pdf",bbox_inches='tight')#,dpi=400)

NameError: name 'max_mass' is not defined